# Controlo da rota de 3 navios num lago infinito

In [2]:
from z3 import *
import numpy as np
from math import cos, sin

In [ ]:
VEL_INIT, VEL_HIGH, VEL_LOW = 0, 10, 1 # velocidade em m/s
THETA = 15 # angulo de viragem em graus
TAU = 0.01 # intervalo de tempo entre estados temporais consecutivos em s

GAUSS_DP = 80 # desvio padrao da gaussiana da inicializacao da posicao em m

In [ ]:
def declare_boat(boat_id):
    """
    A velocidade do state determina qual o seu modo:
        - INIT: vel = VEL_INIT
        - SAFE: vel = VEL_HIGH
        - IMMINENT COLLISION: vel = VEL_LOW
    """
    
    state = {
        "vel": Int(f"b{boat_id}_v"),
        "pos": {"x": Real(f"b{boat_id}_x"), "y": Real(f"b{boat_id}_y")},
        "ang": Int(f"b{boat_id}_a"),
        "time": Real(f"b{boat_id}_t")
    }
    
    return state


def init_boat(state):
    # cond_a = Or([state["ang"] == i*THETA for i in range(360/15)])
    
    # Randomly defined angle
    random_angle = np.random.shuffle([i*THETA for i in range(360/15)])[0]
    cond_a = state["ang"] == random_angle
    
    # Randomly defined position according to gaussian
    cond_x = state["pos"]["x"] == GAUSS_DP * np.random.randn()
    cond_y = state["pos"]["y"] == GAUSS_DP * np.random.randn()
    
    # Deterministic initial definitions
    cond_v = state["vel"] == VEL_INIT
    cond_t = state["time"] == 0
    
    return And(cond_a, cond_v, cond_t)


def untimed_to_imminent(state):
    return state["vel"] == VEL_LOW


def untimed_to_safe(state):
    return state["vel"] == VEL_HIGH


def timed_safe(state_ant, state_atual):
    # Definir evolucao do tempo
    cond_t = state_atual["time"] == state_ant["time"] + TAU
    
    # Definir evolucao da abcissa
    dx = state_ant["vel"] * cos(state_ant["ang"]) * TAU
    cond_x = state_atual["pos"]["x"] == state_ant["pos"]["x"] + dx
    
    # Definir evolucao da ordenada
    dy = state_ant["vel"] * sin(state_ant["ang"]) * TAU
    cond_y = state_atual["pos"]["y"] == state_ant["pos"]["y"] + dy
    
    return And(cond_t, cond_x, cond_y)


def timed_imminent(state_ant, state_atual):
    # Definir evolucao do tempo
    cond_t = state_atual["time"] == state_ant["time"] + TAU
    
    # Definir evolucao da abcissa
    dx = state_ant["vel"] * cos(state_ant["ang"]) * TAU
    cond_x = state_atual["pos"]["x"] == state_ant["pos"]["x"] + dx
    
    # Definir evolucao da ordenada
    dy = state_ant["vel"] * sin(state_ant["ang"]) * TAU
    cond_y = state_atual["pos"]["y"] == state_ant["pos"]["y"] + dy
    
    # Definir evolucao da rota
    theta_pos = state_atual["ang"] == state_ant["ang"] + THETA
    theta_neg = state_atual["ang"] == state_ant["ang"] - THETA
    cond_a = Or(theta_pos, theta_neg)
    
    return And(cond_t, cond_x, cond_y, cond_a)


def imminent_collision():
    

In [ ]:
def declare(i, num_boats=3):
    # Randomly defined initial angles
    angles = [i*THETA for i in range(360/15)]
    random_angles = [np.random.shuffle(angles)[0] for _ in range(num_boats)]
    
    mode = {
        "b1": {"v": VEL_INIT, "a": random_angles[0]},
        "b2": {"v": VEL_INIT, "a": random_angles[1]},
        "b3": {"v": VEL_INIT, "a": random_angles[2]}
    }
    
    state = {
        "b1": {"x": Real(f"tr{i}_b1_x"), "y": Real(f"tr{i}_b1_y")},
        "b2": {"x": Real(f"tr{i}_b2_y"), "y": Real(f"tr{i}_b2_y")},
        "b3": {"x": Real(f"tr{i}_b3_y"), "y": Real(f"tr{i}_b3_y")}
    }
    
    trace = {
        "mode": mode,
        "state": state,
        "time": Real(f"tr{i}_t")
    }
    
    return trace


def init(trace):
    conds = []
    
    # Randomly defined position according to gaussian
    for i in range(len(trace["state"])):
        conds.append(trace["state"][f"b{i+1}"]["x"] == GAUSS_DP * np.random.randn())
        conds.append(trace["state"][f"b{i+1}"]["y"] == GAUSS_DP * np.random.randn())
    
    # Set initial time to 0
    conds.append(trace["time"] == 0)
    
    return And(conds)


def untimed_to_imminent(trace, boat_id1, boat_id2):
    # NOTA: ADICIONAR CONDICOES COLISAO
    
    conds = []
    aux1 = trace["mode"][f"b{boat_id1}"]["a"]
    aux2 = trace["mode"][f"b{boat_id2}"]["a"]
    
    # Mudar o modo
    trace["mode"][f"b{boat_id1}"]["v"] = VEL_LOW
    trace["mode"][f"b{boat_id2}"]["v"] = VEL_LOW
    trace["mode"][f"b{boat_id1}"]["a"] += THETA
    trace["mode"][f"b{boat_id2}"]["a"] += THETA
    
    # Condicao z3 do modo
    conds.append(trace["mode"][f"b{boat_id1}"]["v"] == VEL_LOW)
    conds.append(trace["mode"][f"b{boat_id2}"]["v"] == VEL_LOW)
    conds.append(trace["mode"][f"b{boat_id1}"]["a"] == aux1 + THETA)
    conds.append(trace["mode"][f"b{boat_id2}"]["a"] == aux2 + THETA)
    
    return And(conds)


def untimed_to_safe(trace, boat_id1, boat_id2):
    # NOTA: ADICIONAR CONDICOES COLISAO
    
    conds = []
    
    # Mudar o modo
    trace["mode"][f"b{boat_id1}"]["v"] = VEL_HIGH
    trace["mode"][f"b{boat_id2}"]["v"] = VEL_HIGH
    
    # Condicao z3 do modo
    conds.append(trace["mode"][f"b{boat_id1}"]["v"] == VEL_HIGH)
    conds.append(trace["mode"][f"b{boat_id2}"]["v"] == VEL_HIGH)
    
    return And(conds)              
    

def timed(trace_ant, trace_atual, boat_id):
    # Definir evolucao do tempo
    cond_t = trace_atual["time"] == trace_ant["time"] + TAU
    
    # Definir evolucao da abcissa
    dx = trace_ant["mode"][f"b{boat_id}"]["v"] * cos(trace_ant["mode"][f"b{boat_id}"]["a"]) * TAU
    cond_x = trace_atual["state"][f"b{boat_id}"]["x"] == trace_ant["state"][f"b{boat_id}"]["x"] + dx
    
    # Definir evolucao da ordenada
    dy = trace_ant["mode"][f"b{boat_id}"]["v"] * sin(trace_ant["mode"][f"b{boat_id}"]["a"]) * TAU
    cond_y = trace_atual["state"][f"b{boat_id}"]["y"] == trace_ant["state"][f"b{boat_id}"]["y"] + dy
    
    return And(cond_t, cond_x, cond_y)


def trans(trace):
    
    